In [24]:
#@title Perfil Aerodinàmic

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Parameters of the grid
shape        = (400, 200)
xrange       = (-8, 8)
yrange       = (-3, 5)

plt.rc('font', size = 12)

def plot_wing(alpha_deg = 20):

  alpha = alpha_deg * (np.pi / 180)

  # Parameters of the airfoil
  mu    = -0.15 + 0.15j
  cut_h = 0.07
  V_inf = 3
  R     = np.sqrt((1 - mu.real)**2 + mu.imag**2)
  gamma = 4 * np.pi * V_inf * R * np.sin( alpha + np.arcsin(mu.imag / R) )
  rot   = np.exp(1j * alpha)

  # Inverse Joukowsky transform
  def z2zeta(z):
      discriminant = np.sqrt(z**2 - 4)
      zeta = (z + discriminant) / 2
      in_unit_circle = (np.abs(zeta) <= 1)
      below_wing = (z.real**2 < 4) & (z.imag > 0) & \
                  (z.imag < cut_h * (4 - z.real**2))
      bad_branch = below_wing ^ in_unit_circle
      zeta[bad_branch] -= discriminant[bad_branch]
      return zeta

  # W = vx - i vy
  def complex_velocity(z):
      zeta     = z2zeta(z)
      W_tilde  = V_inf * np.exp(-1j * alpha)
      W_tilde += 1j * gamma / (2 * np.pi * (zeta - mu))
      W_tilde -= V_inf * R**2 * np.exp(1j * alpha) / (zeta - mu)**2
      W = W_tilde / (1 - 1/zeta**2)
      return W.conj()

  # Bernoullis's principle
  def pressure(z):
      st = complex_velocity(z)
      P = 0.5 * (V_inf**2 - np.abs(st)**2)
      return P

  # Define the complex plane grid
  rez  = np.linspace(*xrange, shape[0])[:, None]
  imz  = np.linspace(*yrange, shape[1])[None, :]
  z    = (rez + 1j * imz)

  # Random particles to show flow
  x  = xrange[0] + (xrange[1] - xrange[0]) * np.random.rand(5000)
  y  = yrange[0] + (yrange[1] - yrange[0]) * np.random.rand(5000)
  xy = x + 1j * y

  # Joukowsky airfoil profile
  theta     = np.linspace(0, 2*np.pi, 100)
  zeta_circ = mu + R * np.exp(1j * theta)
  z_airfoil = (zeta_circ + 1/zeta_circ) * rot.conj()

  ##### Do the plotting #####

  fig, ax = plt.subplots(figsize = (10,5))

  ax.set_xlim(*xrange)
  ax.set_ylim(*yrange)

  ax.imshow(pressure(z * rot).T, origin = 'lower', cmap = 'jet', extent = xrange + yrange, vmin = -10, vmax = 7)

  W = complex_velocity(z * rot) / rot
  ax.streamplot(z.real.T, z.imag.T, W.real.T, W.imag.T, color = 'k', linewidth = 0.7)
  ax.fill(z_airfoil.real, z_airfoil.imag, color = 'white', zorder=2)


interact(
    plot_wing,
    alpha_deg = FloatSlider(value=20, min=0, max=45, step=1,
        description="Angle of Attack",
        style={'description_width': '150px', },
        layout={'width': '800px'},
        readout_format='.2f'),
);

interactive(children=(FloatSlider(value=20.0, description='Angle of Attack', layout=Layout(width='800px'), max…